In [2]:
import os
from google.colab import drive
drive.mount("/content/drive")
os.chdir("/content/drive/MyDrive/DP")

Mounted at /content/drive


In [3]:
import pickle
import os, re, sys
import shutil
import nibabel as nib
from scipy.fftpack import fftn, ifftn
import numpy as np
try:
    import matplotlib
    # matplotlib.use("TkAgg")
    import matplotlib.pyplot as plt
    from matplotlib import animation
except:
    print ('matplotlib not imported')


def progress_bar(curr_idx, max_idx, time_step, repeat_elem = "_"):
    max_equals = 55
    step_ms = int(time_step*1000)
    num_equals = int(curr_idx*max_equals/float(max_idx))
    len_reverse =len('Step:%d ms| %d/%d ['%(step_ms, curr_idx, max_idx)) + num_equals
    sys.stdout.write("Step:%d ms|%d/%d [%s]" %(step_ms, curr_idx, max_idx, " " * max_equals,))
    sys.stdout.flush()
    sys.stdout.write("\b" * (max_equals+1))
    sys.stdout.write(repeat_elem * num_equals)
    sys.stdout.write("\b"*len_reverse)
    sys.stdout.flush()
    if curr_idx == max_idx:
        print('\n')

def read_fft_volume(data4D, harmonic=1):
    zslices = data4D.shape[2]
    tframes = data4D.shape[3]
    data3d_fft = np.empty((data4D.shape[:2]+(0,)))
    for slice in range(zslices):
        ff1 = fftn([data4D[:,:,slice, t] for t in range(tframes)])
        fh = np.absolute(ifftn(ff1[harmonic, :, :]))
        fh[fh < 0.1 * np.max(fh)] = 0.0
        image = 1. * fh / np.max(fh)
        # plt.imshow(image, cmap = 'gray')
        # plt.show()
        image = np.expand_dims(image, axis=2)
        data3d_fft = np.append(data3d_fft, image, axis=2)
    return data3d_fft

def save_data(data, filename, out_path):
    out_filename = os.path.join(out_path, filename)
    with open(out_filename, 'wb') as f:
        pickle.dump(data, f, protocol=pickle.HIGHEST_PROTOCOL)
    print ('saved to %s' % out_filename)

def load_pkl(path):
    with open(path) as f:
        obj = pickle.load(f)
    return obj

def imshow(*args,**kwargs):
    """ Handy function to show multiple plots in on row, possibly with different cmaps and titles
    Usage:
    imshow(img1, title="myPlot")
    imshow(img1,img2, title=['title1','title2'])
    imshow(img1,img2, cmap='hot')
    imshow(img1,img2,cmap=['gray','Blues']) """
    cmap = kwargs.get('cmap', 'gray')
    title= kwargs.get('title','')
    axis_off = kwargs.get('axis_off','')
    if len(args)==0:
        raise ValueError("No images given to imshow")
    elif len(args)==1:
        plt.title(title)
        plt.imshow(args[0], interpolation='none')
    else:
        n=len(args)
        if type(cmap)==str:
            cmap = [cmap]*n
        if type(title)==str:
            title= [title]*n
        plt.figure(figsize=(n*5,10))
        for i in range(n):
            plt.subplot(1,n,i+1)
            plt.title(title[i])
            plt.imshow(args[i], cmap[i])
            if axis_off: 
              plt.axis('off')  
    plt.show()
    
def plot_roi(data4D, roi_center, roi_radii):
    """
    Do the animation of full heart volume
    """
    x_roi_center, y_roi_center = roi_center[0], roi_center[1]
    x_roi_radius, y_roi_radius = roi_radii[0], roi_radii[1]
    print ('nslices', data4D.shape[2])

    zslices = data4D.shape[2]
    tframes = data4D.shape[3]

    slice_cnt = 0
    for slice in [data4D[:,:,z,:] for z in range(zslices)]:
      outdata = np.swapaxes(np.swapaxes(slice[:,:,:], 0,2), 1,2)
      roi_mask = np.zeros_like(outdata[0])
      roi_mask[x_roi_center - x_roi_radius:x_roi_center + x_roi_radius,
      y_roi_center - y_roi_radius:y_roi_center + y_roi_radius] = 1

      outdata[:, roi_mask > 0.5] = 0.8 * outdata[:, roi_mask > 0.5]
      outdata[:, roi_mask > 0.5] = 0.8 * outdata[:, roi_mask > 0.5]

      fig = plt.figure(1)
      fig.canvas.set_window_title('slice_No' + str(slice_cnt))
      slice_cnt+=1
      def init_out():
          im.set_data(outdata[0])

      def animate_out(i):
          im.set_data(outdata[i])
          return im

      im = fig.gca().imshow(outdata[0], cmap='gray')
      anim = animation.FuncAnimation(fig, animate_out, init_func=init_out, frames=tframes, interval=50)
      plt.show()

def plot_4D(data4D):
    """
    Do the animation of full heart volume
    """
    print ('nslices', data4D.shape[2])
    zslices = data4D.shape[2]
    tframes = data4D.shape[3]

    slice_cnt = 0
    for slice in [data4D[:,:,z,:] for z in range(zslices)]:
      outdata = np.swapaxes(np.swapaxes(slice[:,:,:], 0,2), 1,2)
      fig = plt.figure(1)
      fig.canvas.set_window_title('slice_No' + str(slice_cnt))
      slice_cnt+=1
      def init_out():
          im.set_data(outdata[0])

      def animate_out(i):
          im.set_data(outdata[i])
          return im

      im = fig.gca().imshow(outdata[0], cmap='gray')
      anim = animation.FuncAnimation(fig, animate_out, init_func=init_out, frames=tframes, interval=50)
      plt.show()

def multilabel_split(image_tensor):
    """
    image_tensor : Batch * H * W
    Split multilabel images and return stack of images
    Returns: Tensor of shape: Batch * H * W * n_class (4D tensor)
    # TODO: Be careful: when using this code: labels need to be 
    defined, explictly before hand as this code does not handle
    missing labels
    So far, this function is okay as it considers full volume for
    finding out unique labels
    """
    labels = np.unique(image_tensor)
    batch_size = image_tensor.shape[0]
    out_shape =  image_tensor.shape + (len(labels),)
    image_tensor_4D = np.zeros(out_shape, dtype='uint8')
    for i in xrange(batch_size):
        cnt = 0
        shape =image_tensor.shape[1:3] + (len(labels),)
        temp = np.ones(shape, dtype='uint8')
        for label in labels:
            temp[...,cnt] = np.where(image_tensor[i] == label, temp[...,cnt], 0)
            cnt += 1
        image_tensor_4D[i] = temp
    return image_tensor_4D

def swapaxes_slv(vol):
    return np.swapaxes(np.swapaxes(vol,0,2),0,1)

def reorder_vol(data):
    ED_GT = swapaxes_slv(data[0][1])
    ED_PD = swapaxes_slv(data[0][2])
    ES_GT = swapaxes_slv(data[1][1])
    ES_PD = swapaxes_slv(data[1][2])
    return (ED_GT, ES_GT, ED_PD, ES_PD)

In [4]:
import os
import time
import nibabel as nib
import pandas as pd
import numpy as np
import scipy
from scipy import ndimage
import matplotlib
# matplotlib.use("TkAgg")
import matplotlib.pyplot as plt
import skimage
from skimage import feature
from scipy import spatial
#from loaders import data_augmentation

def heart_metrics(seg_3Dmap, voxel_size, classes=[3, 1, 2]):
    """
    Compute the volumes of each classes
    """
    # Loop on each classes of the input images
    volumes = []
    for c in classes:
        # Copy the gt image to not alterate the input
        seg_3Dmap_copy = np.copy(seg_3Dmap)
        seg_3Dmap_copy[seg_3Dmap_copy != c] = 0

        # Clip the value to compute the volumes
        seg_3Dmap_copy = np.clip(seg_3Dmap_copy, 0, 1)

        # Compute volume
        # volume = seg_3Dmap_copy.sum() * np.prod(voxel_size) / 1000.
        volume = seg_3Dmap_copy.sum() * np.prod(voxel_size)
        volumes += [volume]
    return volumes

def ejection_fraction(ed_vol, es_vol):
    """
    Calculate ejection fraction
    """
    stroke_vol = ed_vol - es_vol
    return (np.float64(stroke_vol)/np.float64(ed_vol))

def myocardialmass(myocardvol):
    """
    Specific gravity of heart muscle (1.05 g/ml)
    """ 
    return myocardvol*1.05

def bsa(height, weight):
    """
    Body surface Area
    """
    return np.sqrt((height*weight)/3600)

def myocardial_thickness(data_path, slices_to_skip=(0,0), myo_label=2):
    """
    Calculate myocardial thickness of mid-slices, excluding a few apex and basal slices
    since myocardium is difficult to identify
    """
    label_obj = nib.load(data_path)
    myocardial_mask = (label_obj.get_fdata() == myo_label)
    # pixel spacing in X and Y
    pixel_spacing = label_obj.header.get_zooms()[:2]
    assert pixel_spacing[0] == pixel_spacing[1]

    holes_filles = np.zeros(myocardial_mask.shape)
    interior_circle = np.zeros(myocardial_mask.shape)

    cinterior_circle_edge=np.zeros(myocardial_mask.shape)
    cexterior_circle_edge=np.zeros(myocardial_mask.shape)

    overall_avg_thickness= []
    overall_std_thickness= []
    for i in range(slices_to_skip[0], myocardial_mask.shape[2]-slices_to_skip[1]):
        holes_filles[:,:,i] = ndimage.morphology.binary_fill_holes(myocardial_mask[:,:,i])
        interior_circle[:,:,i] = holes_filles[:,:,i] - myocardial_mask[:,:,i]
        cinterior_circle_edge[:,:,i] = feature.canny(interior_circle[:,:,i])
        cexterior_circle_edge[:,:,i] = feature.canny(holes_filles[:,:,i])
        # patch = 64
        # utils.imshow(data_augmentation.resize_image_with_crop_or_pad(myocardial_mask[:,:,i], patch, patch), 
        #     data_augmentation.resize_image_with_crop_or_pad(holes_filles[:,:,i], patch, patch),
        #     data_augmentation.resize_image_with_crop_or_pad(interior_circle[:,:,i], patch,patch ), 
        #     data_augmentation.resize_image_with_crop_or_pad(cinterior_circle_edge[:,:,i], patch, patch), 
        #     data_augmentation.resize_image_with_crop_or_pad(cexterior_circle_edge[:,:,i], patch, patch), 
        #     title= ['Myocardium', 'Binary Hole Filling', 'Left Ventricle Cavity', 'Interior Contour', 'Exterior Contour'], axis_off=True)
        x_in, y_in = np.where(cinterior_circle_edge[:,:,i] != 0)
        number_of_interior_points = len(x_in)
        # print (len(x_in))
        x_ex,y_ex=np.where(cexterior_circle_edge[:,:,i] != 0)
        number_of_exterior_points=len(x_ex)
        # print (len(x_ex))
        if len(x_ex) and len(x_in) !=0:
            total_distance_in_slice=[]
            for z in range(number_of_interior_points):
                distance=[]
                for k in range(number_of_exterior_points):
                    a  = [x_in[z], y_in[z]]
                    a=np.array(a)
                    # print a
                    b  = [x_ex[k], y_ex[k]]
                    b=np.array(b)
                    # dst = np.linalg.norm(a-b)
                    dst = scipy.spatial.distance.euclidean(a, b)
                    # pdb.set_trace()
                    # if dst == 0:
                    #     pdb.set_trace()
                    distance = np.append(distance, dst)
                distance = np.array(distance)
                min_dist = np.min(distance)
                total_distance_in_slice = np.append(total_distance_in_slice,min_dist)
                total_distance_in_slice = np.array(total_distance_in_slice)

            average_distance_in_slice = np.mean(total_distance_in_slice)*pixel_spacing[0]
            overall_avg_thickness = np.append(overall_avg_thickness, average_distance_in_slice)

            std_distance_in_slice = np.std(total_distance_in_slice)*pixel_spacing[0]
            overall_std_thickness = np.append(overall_std_thickness, std_distance_in_slice)

    # print (overall_avg_thickness)
    # print (overall_std_thickness)
    # print (pixel_spacing[0])
    return (overall_avg_thickness, overall_std_thickness)

In [51]:
import os, re
import numpy as np
import pandas as pd
import sys
# print sys.path
sys.path.append("../") 
# Custom 

HEADER = ["Name", "ED[vol(LV)]", "ES[vol(LV)]", "ED[vol(RV)]", "ES[vol(RV)]",
          "ED[mass(MYO)]", "ES[vol(MYO)]", "EF(LV)", "EF(RV)", "ED[vol(LV)/vol(RV)]", "ES[vol(LV)/vol(RV)]", "ED[mass(MYO)/vol(LV)]", "ES[vol(MYO)/vol(LV)]",
           "ES[stdev(mean(MWT|SA)|LA)]", "ES[mean(stdev(MWT|SA)|LA)]", "ES[stdev(stdev(MWT|SA)|LA)]", 
           "ED[stdev(mean(MWT|SA)|LA)]", "ED[mean(stdev(MWT|SA)|LA)]", "ED[stdev(stdev(MWT|SA)|LA)]", "GROUP"]

def calculate_metrics_for_training(data_path_list, name='train'):

	res = []
	for data_path in data_path_list:
		patient_folder_list = os.listdir(data_path)
		# print (patient_folder_list)
		for patient in patient_folder_list:
			patient_folder_path = os.path.join(data_path, patient)
			config_file_path = os.path.join(patient_folder_path, 'Info.cfg')
			patient_data = {}
			with open(config_file_path) as f_in:
				for line in f_in:
					l = line.rstrip().split(": ")
					patient_data[l[0]] = l[1]

			# Read patient Number
			m = re.match("patient(\d{3})", patient)
			patient_No = int(m.group(1))
			# Read Diastole frame Number
			ED_frame_No = int(patient_data['ED'])
			ed_img = "patient%03d_frame%02d.nii.gz" %(patient_No, ED_frame_No)
			# Read Systole frame Number
			ES_frame_No = int(patient_data['ES'])
			es_img = "patient%03d_frame%02d.nii.gz" %(patient_No, ES_frame_No)

			pid = 'patient{:03d}'.format(patient_No)
			# Load data
			ed_data = nib.load(os.path.join(data_path, pid, ed_img))
			es_data = nib.load(os.path.join(data_path, pid, es_img))

			ed_lv, ed_rv, ed_myo = heart_metrics(ed_data.get_fdata(),
			                ed_data.header.get_zooms())
			es_lv, es_rv, es_myo = heart_metrics(es_data.get_fdata(),
			                es_data.header.get_zooms())
			ef_lv = ejection_fraction(ed_lv, es_lv)
			ef_rv = ejection_fraction(ed_rv, es_rv)

      # ef_lv = (ed_lv-es_lv)/ed_lv
      # ef_rv = (ed_rv-es_rv)/es_rv

			myo_properties = myocardial_thickness(os.path.join(data_path, pid, es_img))
			# es_myo_thickness_max_avg = np.amax(myo_properties[0])
			es_myo_thickness_std_avg = np.std(myo_properties[0])
			es_myo_thickness_mean_std = np.mean(myo_properties[1])
			es_myo_thickness_std_std = np.std(myo_properties[1])

			myo_properties = myocardial_thickness(os.path.join(data_path, pid, ed_img))
			# ed_myo_thickness_max_avg = np.amax(myo_properties[0])
			ed_myo_thickness_std_avg = np.std(myo_properties[0])
			ed_myo_thickness_mean_std = np.mean(myo_properties[1])
			ed_myo_thickness_std_std = np.std(myo_properties[1])
			# print (es_myo_thickness_max_avg, es_myo_thickness_std_avg, es_myo_thickness_mean_std, es_myo_thickness_std_std,
			#      ed_myo_thickness_max_avg, ed_myo_thickness_std_avg, ed_myo_thickness_std_std, ed_myo_thickness_std_std)



			heart_param = {'EDV_LV': ed_lv, 'EDV_RV': ed_rv, 'ESV_LV': es_lv, 'ESV_RV': es_rv,
			       'ED_MYO': ed_myo, 'ES_MYO': es_myo, 'EF_LV': ef_lv, 'EF_RV': ef_rv, 
             'ES_MYO_STD_AVG_T': es_myo_thickness_std_avg, 'ES_MYO_AVG_STD_T': es_myo_thickness_mean_std, 
             'ES_MYO_STD_STD_T': es_myo_thickness_std_std, 'ED_MYO_STD_AVG_T': ed_myo_thickness_std_avg, 
             'ED_MYO_AVG_STD_T': ed_myo_thickness_mean_std, 'ED_MYO_STD_STD_T': ed_myo_thickness_std_std,}
			r=[]

			r.append(pid)
			r.append(heart_param['EDV_LV'])
			r.append(heart_param['ESV_LV'])
			r.append(heart_param['EDV_RV'])
			r.append(heart_param['ESV_RV'])
			r.append(heart_param['ED_MYO'])
			r.append(heart_param['ES_MYO'])
			r.append(heart_param['EF_LV'])
			r.append(heart_param['EF_RV'])
			r.append(ed_lv/ed_rv)
			r.append(es_lv/es_rv)
			r.append(ed_myo/ed_lv)
			r.append(es_myo/es_lv)
			# r.append(patient_data[pid]['Height'])
			# r.append(patient_data[pid]['Weight'])
			# r.append(heart_param['ES_MYO_MAX_AVG_T'])
			r.append(heart_param['ES_MYO_STD_AVG_T'])
			r.append(heart_param['ES_MYO_AVG_STD_T'])
			r.append(heart_param['ES_MYO_STD_STD_T'])

			# r.append(heart_param['ED_MYO_MAX_AVG_T'])
			r.append(heart_param['ED_MYO_STD_AVG_T'])
			r.append(heart_param['ED_MYO_AVG_STD_T'])
			r.append(heart_param['ED_MYO_STD_STD_T'])
			r.append(patient_data['Group'])
			res.append(r)

		df = pd.DataFrame(res, columns=HEADER)
		if not os.path.exists('./training_data'):
			os.makedirs('./training_data')	
		df.to_csv("./training_data/parameters_{}.csv".format(name), index=False)

if __name__ == '__main__':
	#Path to ACDC training set
	train_path = ['/content/drive/MyDrive/DP/dataset/train_set/']
	validation_path = ['/content/drive/MyDrive/DP/dataset/validation_set/']
	full_train = ['/content/drive/MyDrive/DP/dataset/train_set/', '/content/drive/MyDrive/DP/dataset/validation_set/']
	calculate_metrics_for_training(train_path, name='train')
	calculate_metrics_for_training(validation_path, name='validation')
	calculate_metrics_for_training(full_train, name='training')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:41: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:263: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims, where=where)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:223: RuntimeWarning: invalid value encountered in true_divide
  subok=False)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:254: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3441: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:86: RuntimeWarning: invalid value encountered in double_scalars
/us

In [25]:
data = nib.load("/content/drive/MyDrive/DP/dataset/train_set/patient001/patient001_frame01.nii.gz")
data2 = nib.load("/content/drive/MyDrive/DP/dataset/train_set/patient001/patient001_frame12.nii.gz")
ed_lv, ed_rv, ed_myo = heart_metrics(data.get_fdata(),data.header.get_zooms())

In [26]:
es_lv, es_rv, es_myo = heart_metrics(data2.get_fdata(),data2.header.get_zooms())

In [30]:
ejection_fraction(ed_lv, es_lv)

0.051679405371292335

In [5]:
data3 = nib.load("/content/drive/MyDrive/DP/dataset/train_set/patient056/patient056_frame01.nii.gz")
data4 = nib.load("/content/drive/MyDrive/DP/dataset/train_set/patient003/patient003_frame15_gt.nii.gz")
heart_metrics(data3.get_fdata(),data3.header.get_zooms())

[0.0, 0.0, 0.0]

In [23]:
heart_metrics(data4.get_fdata(),data4.header.get_zooms())

[241088.8671875, 174584.9609375, 201074.21875]

In [47]:
np.amax(myocardial_thickness("/content/drive/MyDrive/DP/dataset/train_set/patient001/patient001_frame12.nii.gz")[0])

3.2646946637892276